In [2]:
import numpy as np
from MetaLearner import *

In [3]:
import pickle


In [ ]:
# todo: https://stackoverflow.com/questions/24168569/how-to-add-names-to-a-numpy-array-without-changing-its-dimension

In [ ]:
# This file runs the experiment

In [4]:
tf.keras.backend.set_floatx('float64')

In [5]:
file_name = "/Users/arberimbibaj/Documents/Master Thesis ETH/DataSets /Generated/data_synthetic_500_1000.pkl"
open_file = open(file_name, "rb")
data = pickle.load(open_file)
open_file.close()

In [6]:
n_setups = 2
sample_sizes = [500, 1000]
n_runs = 2

In [ ]:
learners = [TLearner('rf'), SLearner('rf'), XLearner('rf'), RLearner('rf'), DRLearner('rf'), RALearner('rf'),
            PWLearner('rf'), ULearner('rf'),
            TLearner('lasso'), SLearner('lasso'), XLearner('lasso'), RLearner('lasso'), DRLearner('lasso'),
            RALearner('lasso'), PWLearner('lasso'), ULearner('lasso'),
            TLearner('nn'), SLearner('nn'), XLearner('nn'), RLearner('nn'), DRLearner('nn'), RALearner('nn'),
            PWLearner('nn'), ULearner('nn')]

In [7]:
learners = [TLearner('nn'), SLearner('nn'), XLearner('nn'), RLearner('nn'), DRLearner('nn'), RALearner('nn'),
            PWLearner('nn'), ULearner('nn')]

In [ ]:
# What we want: For each setting and base learner --> matrix which includes the MSE, columns: Metalearners + SampleSize, rows: Run(1-10)
# --> So we can plot the mean and 1se of the MSE for each Metalearner, each samplesize; giving the setting and baselearner
# --> number of plots: settings*baselearner.

In [8]:
results = []

for i in range(n_setups):
    results.append([])

for i in range(n_setups):
    for baselearner in range(1):
        results[i].append([])

results

[[[]], [[]]]

In [9]:
dim = data[0][0][0][0].shape[1]  # dimension of x


# helper function to get y, x, w, tau, out of data[][][][]
def get_variables(dataset, setup, samplesize, run, train_test):
    y = dataset[setup][samplesize][run][train_test][:, 0]
    x = dataset[setup][samplesize][run][train_test][:, 1:(dim - 2)]
    w = dataset[setup][samplesize][run][train_test][:, (dim - 2)]
    tau = dataset[setup][samplesize][run][train_test][:, (dim - 1)]
    return y, x, w, tau

In [10]:
b = 0
m = 0
s = 0

for i in range(n_setups):
    print(f'Setup: {i + 1}')
    # array for all mses for one setup
    setup_mse = np.empty(shape=(0, 24))
    s = 0  # restart index for samplesize
    for size in sample_sizes:
        print(f'Sample Size: {s + 1}')
        # array for all mses for one setup and samplesize.
        size_mse = np.empty(shape=(0, 24))
        for r in range(n_runs):
            print(f'Run: {r + 1}')
            # array for all mses in one setup, samplesize and run.
            mses = np.empty(shape=(1, 24))
            # get data for specific setup, samplesize and run.
            temp_y_train, temp_x_train, temp_w_train, temp_tau_train = get_variables(dataset=data, setup=i,
                                                                                     samplesize=s, run=r,
                                                                                     train_test=0)
            temp_y_test, temp_x_test, temp_w_test, temp_tau_test = get_variables(dataset=data, setup=i,
                                                                                 samplesize=s, run=r,
                                                                                 train_test=1)
            # restart index for metalearner
            m = 0
            for l in learners:
                if m % 8 == 0: print(f'BaseLearner {m / 8 + 1}')
                # training and testing MetaLearner.
                learner = l
                learner.fit(temp_x_train, temp_y_train, temp_w_train)
                predictions = learner.predict(temp_x_test)
                temp_mse = ((predictions - temp_tau_test) ** 2).mean()
                # append mse of specific metalearner to 'mses'.
                mses[0, m] = temp_mse
                # update index
                m += 1
            # append 'mses' to 'size_mse'.
            size_mse = np.append(size_mse, mses, axis=0)
        # append 'size_mse' to 'setup_mse'.
        setup_mse = np.append(setup_mse, size_mse, axis=0)
        # update index
        s += 1
    # append to results
    results[i][0] = setup_mse[:, 0:8]  # random forest
    # results[i][1] = setup_mse[:, 8:16]  # lasso
    # results[i][2] = setup_mse[:, 16:24]  # neural network





# this should make sense i guess! :)
# TODO: CHECK IF IT MAKES IT FASTER IF EVERYTHING IS EITHER IN NP OR IN LIST/ARRAY BASE PYTHON.
# TODO: DONE :)
# TODO: DONE :)
# TODO: DONE :)

### IT TOOK 6M 40S FOR --> SAMPLE SIZES [200, 200], TEST SIZE = 50, 2 SETUPS, 2 RUNS LOL

Setup: 1
Sample Size: 1
Run: 1
BaseLearner 1.0
Run: 2
BaseLearner 1.0
Sample Size: 2
Run: 1
BaseLearner 1.0
Run: 2
BaseLearner 1.0
Setup: 2
Sample Size: 1
Run: 1
BaseLearner 1.0
Run: 2
BaseLearner 1.0
Sample Size: 2
Run: 1
BaseLearner 1.0
Run: 2
BaseLearner 1.0


In [27]:
results[0][0]

array([[9.79529157e-01, 1.89357319e-01, 6.91835179e-01, 1.26446844e+00,
        4.79406046e+00, 1.27244946e+00, 3.18679463e+02, 1.08102732e+00],
       [5.59491546e-01, 2.03706497e-01, 9.27346812e-01, 2.41689041e+00,
        2.04387810e-01, 1.45049069e+00, 7.41109178e+01, 2.52662918e+00],
       [2.87717515e-01, 1.04779995e-01, 4.18623459e-01, 1.60711967e+00,
        1.15509054e+00, 6.93351046e-01, 1.01837731e+02, 1.35955499e+00],
       [7.55225911e-01, 1.50240740e-01, 5.35911274e-01, 1.14435681e+00,
        5.15420931e-02, 9.84803196e-01, 4.89825079e+02, 1.06982208e+00]])

In [ ]:
# results[0][0]
# 0.979529, 0.189357, 0.691835, 1.264468, 4.794060, 1.272449, 318.679463,   1.081027
# 0.559492, 0.203706, 0.927347, 2.416890, 0.204388, 1.450491, 74.110918,    2.526629
# 0.287718, 0.104780, 0.418623, 1.607120, 1.155091, 0.693351, 101.837731,   1.359555
# 0.755226, 0.150241, 0.535911, 1.144357, 0.051542, 0.984803, 489.825079,   1.069822


# --> settings 0,1, size 500,1000, test size 500, only neural networks
# --> look nice :)
# --> only the PW Learner looks bad :(

In [ ]:
# Results[1][0]
# 2.541785,0.372843,3.078384,7.408911,0.083564,5.127290,97.172635,6.231679
# 2.327750,0.157416,2.486625,5.897240,1.015858,4.196150,71.799654,5.595057
# 2.553822,0.429779,2.908590,3.492300,0.148840,3.881968,28.495289,3.088685
# 3.231855,0.775711,3.748154,4.025924,0.132979,5.174252,25.184674,4.476805


In [ ]:
data[0][0][0][0]

In [ ]:
# 0.928021
#
#
#
